# MATH20014 Mathematical Programming: Group 3 Project Submission 

Group Members: Hamad Babur, Ryan Lam, Jiawei Li, Sherry Shen 


## Table of Contents

- [Introduction](#Intro)
- Part A: [Introduction to `PyGame`](#PartA)
    - Section 1: [Animating Ball Motion](#S1)
    - Section 2: [Sierpenski Triangle](#S2)
    - Section 3: [Fractals and Trees](#S3)
- Part B: [Applications of `Pygame`](#PartB)
    - Section 4: [Julia and Mandelbrot Sets](#S4)
        - 4.1: [Animation of Julia Sets](#S4.1)
        - 4.2: [Animation of Mandelbrot Sets](#S4.2)
    - Section 5: [Tower of Hanoi](#S5)
        - 5.1 [Definitions and Objectives](#S5.1)
        - 5.2 [Producing a Solution](#S5.2)
        - 5.3 [Identifying Valid Moves](#S5.3)
        - 5.4 [Animation of the Puzzles](#S5.4)
        - 5.5 [Extensions](#S5.5)
- [Conclusion](#Conc)
- [References and Bibilography](#Bib)

## Introduction <a name='Intro'></a>


Recursion is one of the most powerful tools in programming. It is not only a rather intuitive mathematical concept, but it is also easy to implement in any programming language. 

In this Python project, we shall both visually and mathematically appreciate the power of recursion.  We shall focus on a set of modules collectively known as `PyGame` [(Shinners, 2021)](#PyGame_Intro). They provide a concise way for us to produce interactive animations within Python. 

The language of recursion naturally leads to the study of *fractals*, that is, self-repeating patterns and geometric objects. As we shall see later, a lot of patterns in nature are also closely related to fractals. 


The first part of the project consists of three short parts, aiming to understand the power of the `PyGame` module. 

Firstly, in section 1, we shall introduce and develop a simple PyGame animation function bouncing_ball, in which one or more balls bounce in a square box in two dimensions with various constraints. 

In section 2, we shall introduce fractals through the Sierpinski triangle and make an animation of how the Sierpinski triangles are produced. 

In section 3, we shall explore a particular construction and animation of a recursively defined “tree”. 

The second part of the project shall focus on some interesting applications when we shall integrate PyGame with different programming techniques. 

In section 4, we shall look at Julia sequences and we shall animate the Julia set as a transition from one picture to another. In addition to this, we shall look at Mandelbrot sequences, and then we shall create a program that zooms in and out of the Mandelbrot set. 

Finally, in section 5, we shall explore a classical puzzle - the Tower of Hanoi. We shall briefly look at the solution, the animation, and other time complexity issues.

In [1]:
# Required Imports:
import pygame, os, timeit
import matplotlib.pyplot as plt
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


<a id='PartA'></a>
## Part A: Introduction to `PyGame`


In this part, we shall demonstrate animating and drawing in PyGame, which is very useful for our applications in [part B](#PartB).

<a id='S1'></a>
### Section 1: Animating Ball Motion 


In this section, we will include some brief code to outline the main functionality of PyGame.
For this, recall that we are given a pygame animation function bouncing_ball, and we will further develop it such that:

1.	The user should be able to change the original position of the ball and should be able to change the speed of the ball.
2.	The ball should slow down under the effect of gravity.
3.	Two or more balls bounce within the same square (and of one another).


To do this, we can utilize the power of classes in Python. We can observe that intuitively, a moving ball should have a size, its current position and its current velocity, and we can make the following definition:


In [2]:
class Ball(object):
    '''
    The class Ball encapsulate all the information needed to 
    draw a ball on the PyGame module.
    '''
    def __init__(self, x, y, x_step, y_step, size=50):
        '''
        Initialization of the class Ball. 
        (x, y) - the coordinate relative to the screen, 
        both of them are float from 0 to 1.
        (x_step, y_step) - the velocity of the ball (pixel per frame).
        size - optional argument, size of the ball.
        '''
        self.x = x
        self.y = y
        self.x_step = x_step
        self.y_step = y_step
        self.size = size

Note that we set the attribute `size` as optional, as this would simplify the initialization of the object a bit.

Now we are ready to adapt the function `bouncing_ball` into `bouncing_ball_ext` such that gravity is implemented for one bouncing ball. 

Note that in `Pygame`, a coordinate grid is implemented. And the $y$-axis $y=0$ is at the top of the screen, that’s why when gravity is negative, the ball slows down at the top.

Before we create a function that animates a Julia set, we need to create a "recipe" for the function.


> **Note:** A click of a mouse button or pressing of a keyboard button consists of the following two actions.
> 1. Pressing down on a button, and
>
> 2. Releasing a button.
>
> Both of these are stipulated by different events in PyGame.

We shall only be using the first action described in Definition 6.1 to describe a press of the keyboard in this particular function. This is because, for this function, the only time the keyboard is used is when a button/key is being pressed.

Firstly, the program needs to load the files, and we have a function called `julia_export` to do this. Then we need to initialise PyGame with an appropriate screen size and set the time clock. Then, we need to load the first picture, and wait for a few seconds and then load the next picture, and this goes on until the last picture. Then we need to add in extra features like pausing and speeding up and down the animation. The steps below describe on what the program should do in detail.

> 1. Export images using julia_export.
>
> 2. Configure screen size and colour as tuples, and store the clock time in a variable.
>
> 3. Initialise Pygame, and set the program window using the screen size tuple used earlier and the title.
>
> 4. Load the picture JuliaPic1.png above a white background and strech it to the program window.
>
> 5. Use the current speed factor and the clock time to wait for a specified amount of time, and then load the next picture, replacing the current picture.
>
> 6. Repeat Step 5 until the last image loads, then go back to the first image and go back to Step 5.

In addition to this, the program needs to know how to stop the animation, speed up the animation and slow down the animation. In order to do this, we shall use the following principles:

> - If the user initiates the exit sequence, stop running the program and quit the PyGame window.
>
> - If the user presses the Space button, load the current picture rather than loading the next picture in Step 5 to create a static image.
>
> - If the user presses the Up Arrow button, increase the speed factor to speed up Step 5 and as a result, this speeds up the animation.
>
> - If the user presses the Down Arrow button, decrease the speed factor to slow down Step 5 and as a result, this slows down the animation.

Using the aforementioned "recipe", we shall now create a function that animates the
 Julia set.


In [3]:
def bouncing_ball_ext(ball, speed_factor=5,gravity=9.81):
    '''
    Play a bouncing ball animation according to the inputs:
    Inputs:
    ball - an object from the class Ball
    speed_factor - the frames per second of animation
    gravity - the drag factor affecting the motion of the ball (positive is upwards)
    (N.B., gravity = 0 corresponds to no gravity, obviously)
    Output: None
    '''
    # Required variables, x0, y0 is the starting coordinate of the ball
    screen_size = (screen_width, screen_height) = (1080, 800)
    white = (255,255,255)
    x0, y0 = (screen_width - ball.size)*ball.x, (screen_height - ball.size)*ball.y
    g = -gravity
    # Used for the pause time in the animation while loop below
    frames_per_second = 10 + 10*speed_factor
    clock = pygame.time.Clock()
    # Set up the animation     
    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    # Titles and Instructions
    caption = f'Bouncing Ball with Two Balls with gravity factor {-gravity}'
    caption += '                              '
    caption += '(Keystroke:  \'Space\' to start or pause, \'ArrowUp/ArrowDown\' to speed up/down)'
    pygame.display.set_caption(caption)
    # Load the image
    ball_fig = pygame.image.load("intro_ball.gif")
    ball_fig = pygame.transform.scale(ball_fig, (ball.size, ball.size))
    # Assign an pygame.Rect object to the ball
    ball_rect = pygame.Rect(x0,y0,ball.size,ball.size)
    # Initalize the Screen
    screen.fill(white)
    screen.blit(ball_fig, ball_rect)
    pygame.display.flip()
    # Variable to detect change
    keep_running = True
    move_ball = False
    # Animation loop 
    while keep_running:
        # If a keyboard event happens register it... 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                keep_running = False
            # Start/Stop Animation
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                move_ball = not move_ball
            # Speed change if ArrowUp/ArrowDown is pressed
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
        if move_ball:
            # Move the ball a step under gravity
            ball.y_step += g/(frames_per_second)
            ball_rect.x += ball.x_step
            ball_rect.y += ball.y_step
            if ball_rect.left < 0 or ball_rect.right > screen_width:
                ball.x_step = - ball.x_step
            if ball_rect.top < 0 or ball_rect.bottom > screen_height:
                ball.y_step = - ball.y_step
        # Redraw and update the screen
        screen.fill(white)
        screen.blit(ball_fig, ball_rect)
        pygame.display.flip()
        clock.tick(frames_per_second)
    pygame.quit()
    return None 

We can test our function using one instance of our class `Ball`.

In [4]:
# A ball positioned at the middle, with velocity (1.2, 3).
ballA = Ball(0.5, 0.5, 1.2, 3)
bouncing_ball_ext(ballA)

Next, we can implement collision between two balls. 

We assume every collision is *elastic*, that is,  we only have to exchange the velocity between the two balls when a collision is detected.

In [5]:
def bouncing_ball_two(ball1, ball2, speed_factor=5,gravity=9.81):
    '''
    Play an animation similar to bouncing_ball with two balls with elastic collision between the balls.
    The input (and code, actually) are similar with an extra parameter ball2 in the Class Ball.
    '''
    # Required variables, x0, y0 is the starting coordinate of the ball
    screen_size = (screen_width, screen_height) = (1080, 800)
    white = (255,255,255)
    x0, y0 = (screen_width - ball1.size)*ball1.x, (screen_height - ball1.size)*ball1.y
    x1, y1 = (screen_width - ball1.size)*ball2.x, (screen_height - ball2.size)*ball2.y
    g = -gravity
    # Used for the pause time in the animation while loop below
    frames_per_second = 10 + 10*speed_factor
    clock = pygame.time.Clock()
    # Set up the animation     
    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    # Titles and Instructions
    caption = f'Bouncing Ball with Two Balls with gravity factor {-gravity}'
    caption += '                              '
    caption += '(Keystroke:  \'Space\' to start or pause)'
    pygame.display.set_caption(caption)
    # Load the image and rescale
    ball1_fig = pygame.image.load("intro_ball1.gif")
    ball1_fig = pygame.transform.scale(ball1_fig, (ball1.size, ball1.size))
    ball1_rect = pygame.Rect(x0,y0,ball1.size,ball1.size)
    ball2_fig = pygame.image.load("intro_ball2.gif")
    ball2_fig = pygame.transform.scale(ball2_fig, (ball2.size, ball2.size))
    ball2_rect = pygame.Rect(x1,y1,ball2.size,ball2.size)
    # Initalize the Screen
    screen.fill(white)
    screen.blit(ball1_fig, ball1_rect)
    screen.blit(ball2_fig, ball2_rect)
    pygame.display.flip()
    # Variable to detect change
    keep_running = True
    move_ball = False
    # Animation loop 
    while keep_running:
        # Keyboard Event: Quit Button and Space to Start/Stop
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                keep_running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                move_ball = not move_ball
            # Speed change if ArrowUp/ArrowDown is pressed
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
        if move_ball:
            ball1.y_step += g/(frames_per_second)
            ball2.y_step += g/(frames_per_second)
            # Move the ball a step and Screen Border Detection
            ball1_rect.x += ball1.x_step
            ball1_rect.y += ball1.y_step
            if ball1_rect.left < 0 or ball1_rect.right > screen_width:
                ball1.x_step = - ball1.x_step
            if ball1_rect.top < 0 or ball1_rect.bottom > screen_height:
                ball1.y_step = - ball1.y_step
            ball2_rect.x += ball2.x_step
            ball2_rect.y += ball2.y_step
            if ball2_rect.left < 0 or ball2_rect.right > screen_width:
                ball2.x_step = - ball2.x_step
            if ball2_rect.top < 0 or ball2_rect.bottom > screen_height:
                ball2.y_step = - ball2.y_step
            # Collision Detection 
            collide = ball1_rect.colliderect(ball2_rect)
            if collide:
                # If they collide, the velocity of the two balls exchange
                copy_ball1 = ball1
                ball1 = Ball(ball2.x, ball2.y, ball2.x_step, ball2.y_step)
                ball2 = Ball(copy_ball1.x, copy_ball1.y, copy_ball1.x_step , copy_ball1.y_step) 
        # Redraw and reinitiate              
        screen.fill(white)
        screen.blit(ball1_fig, ball1_rect)
        screen.blit(ball2_fig, ball2_rect)
        pygame.display.flip()
        clock.tick(frames_per_second)
    pygame.quit()
    return None 

In [6]:
# Testing Cell
ballA = Ball(0.5, 0.5, 1.2, 3, 70)
ballB = Ball(0.5, 0.2, 1, -5, 100)
bouncing_ball_two(ballA, ballB, gravity =0, speed_factor =20)

<a id='S2'></a>
### Section 2: Sierpenski Triangle 


Animating fractal constructions gives us a direct insight into how the function generating the fractal operates recursively. A typical example is the *Sierpinski Triangle*. It is a self-similar fractal, for which can be created by starting with one large, equilateral triangle, and then repeatedly cutting smaller triangles out of its centre. Interested readers are referred to (Falconer, 1990) for details. 

We are given a simple `PyGame` animation `draw_sierpinski`. Note that this function depends on `make_sierpinski`, which calls on itself using depth-1 recursively. Similar constructs shall appear in our next section while drawing a recursive tree as well.

We shall improve and develop on this basis to meet the following three conditions:

1.	The user can be able to choose the depth of the triangle and stop and start the animation.
2.	The user is also able to change the speed of the animation.
3.	Adding colours (in RGB value) to the triangles and the background.


An interactive version for testing is given below:

<a id='S3'></a>
### Section 3: Fractals and Trees 

In this section, we shall visualize constructing a fractal tree as outlined in Chapter 16 of (Mandelbrot, 1983). 

We shall first outline how we are going to construct a **tree of depth $n$** for any non-negative integer $n$. 

We start with a straight line called a **trunk at depth 0**.  The trunk splits into three lines, which we call **branches**. The left branch and right branch make an angle $\theta$ with the central branch, which is constructed by extending the trunk at depth 0. Then, all three branches can be considered as a **trunk of depth 1**, and each of them can split into three branches (i.e. trunk at depth 2) and so on. 

TODO: What is a tree?

Note that we employ a recursive definition of the tree, and the tree should have $3^n$ trunks.


Then, we shall use `PyGame` to create an animation of drawing the recursive tree. Meanwhile, four conditions need to be satisfied, and this is of the following:

1.	Users should be able to change the depth of the tree.

2.	Users should be able to start and stop the tree.

3.	The leaves should be coloured green and the trunk should be coloured brown.

4.	Users should be able to change the speed of drawing the tree.


<a id='PartB'></a>
## Part B: Applications of `PyGame` 

In this part, we shall focus on two major applications of `PyGame`, namely exploring complex function dynamics in section 4, and a famous recreational puzzle in section 5.

<a id='S4'></a>
### Section 4: Julia and Mandelbrot Sets 

TODO: Add introduction before parts.


We shall fix a parameter $j_p \in \mathbb{C}$. Hence, the **Julia set with parameter $\mathit{j_p}$** is defined as the values of $c$ for which the following recurrence relation is bounded for all $(z_n)_{n \geq 0}$:

$$
z_0 = c  \quad \text{and} \quad z_n = z_{n-1}^2 + j_p, \quad \text{for } n = 1,2,3,\dots
$$

We say that $(z_n)_{n \geq 0}$ is the Julia sequence with parameter $j_p$ induced by $c$. From this, we can see that $(z_n)_{n \geq 0}$ only relies on $c$ and the fixed parameter $j_p$.

Firstly, need to know as to what a Mandelbrot set is. The **Mandelbrot set** is defined as the values of $c$ for which the following recurrence relation is bounded for all $(z_n)_{n \geq 0}$:

$$
z_0 = 0  \quad \text{and} \quad z_n = z_{n-1}^2 + c, \quad \text{for } n = 1,2,3,\ldots
$$

We say that $(z_n)_{n \geq 0}$ is the Mandelbrot sequence induced by $c$. From this, we can see that $(z_n)_{n \geq 0}$ only relies on $c$.

<a id='S4.1'></a> 
#### 4.1 Animation of Julia Sets


Firstly, we shall look at Julia sets and animate images of Julia sets as transistioning slides.

We shall now create a function `julia` that calculates the largest term `n_max` for which each term $z_n$ of the corresponding Julia sequence with parameter $j_p$ induced by $c$ is bounded by $2$, otherwise the sequence $(z_n)_{n \geq 0}$ diverges as $n$ tends to infinity. To avoid long processing times, we shall set a maximum number of terms `max_n` that is looped through in the function and if it is reached, the function returns the value of `max_n`.

In [7]:
def julia(j_p, c, max_n):
    '''
    The julia function calculates the largest number n_max
    for which each term z_n of the corresponding Julia sequence
    with parameter j_p induced by c is bounded by 2. 
    '''
    # Set the "zeroth" term as c.
    z = c
    # Set the largest number as 0.
    n_max = 0
    # Iterate until term is not bounded by 2.
    while abs(z) <= 2:
        # Derive the next term of the sequence from the current term.
        z = (z ** 2) + j_p
        # Increment the largest number n_max by 1.
        n_max += 1
        # Break the loop early when the largest number n_max reaches 
        # the maximum number of iterations max_n.
        if n_max == max_n:
            break
    # Return the value of the largest number n_max.   
    return n_max

We shall now create data for our Julia set. In order to do this, we need to create vectors `x_vector` and `y_vector` in order to create grid matrices `x_matrix` and `y_matrix` as input matricies. The vectors must be a number `c_intervals` of equally spaced intervals between the values `c_min` and `c_max`. 



We shall create the following function that takes a number `c_intervals` of equally spaced intervals between the values `c_min` and `c_max` and calculates `x_matrix` and `y_matrix`, and inputs the matrices into the `julia` function to create an output matrix `z_matrix`. In order to generate data needed for the set, we need `x_matrix`, `y_matrix` and `z_matrix`.

In [8]:
def julia_data(j_p, max_n, c_min, c_max, c_intervals):
    '''
    The julia_data function calculates the matrices 
    required for graphing the julia function.
    '''
    # Calculate the vectors required for the two matrices required for input.
    x_vector = np.linspace(c_min, c_max, c_intervals)
    y_vector = np.linspace(c_min, c_max, c_intervals)
    x_matrix, y_matrix = np.meshgrid(x_vector, y_vector)
    # Create a zero output matrix with the same dimensions as the input matrices.
    z_rows, z_cols = np.shape(x_matrix)
    z_matrix = np.zeros((z_rows, z_cols), dtype = int)
    # Calculate every element of the output matrix for every 
    # element in the input matrix using the julia function.
    for i in range(c_intervals):
        for j in range(c_intervals):
            c = x_matrix[i, j] + y_matrix[i, j] * 1j
            z_matrix[i, j] = julia(j_p, c, max_n)
    return x_matrix, y_matrix, z_matrix

We shall create a folder, so that we can export the images into an appropriate directory entitled "JULIA_GRAPHS".

In [9]:
# Obtain the current working directory.
currwd1 = str(os.getcwd())
# Create a subdirectory within the current working directory using the system terminal.
createdir1 = "mkdir \"{}//JULIA_GRAPHS\"".format(currwd1)
os.system(createdir1)

1

We shall now create a function that generates and exports the Julia set.

In [10]:
def julia_graphpic(j_p, max_n, c_min, c_max, c_intervals, 
plt_width, plt_height, filename):
    '''
    The julia_graphpic function creates a Julia graph and exports
    the Julia graph as a picture with default resuloutions.
    '''
    # Calculate the two input matrices and one output matrix using the julia_data function.
    x_matrix, y_matrix, z_matrix = julia_data(j_p, max_n, c_min, c_max, c_intervals)
    # Produce a Julia graph from the given matrices without the axes.
    plt.figure(figsize = (plt_width, plt_height))
    plt.pcolor(x_matrix, y_matrix, z_matrix, cmap = 'plasma', vmin = 0, vmax = max_n, shading = "auto")
    plt.axis("off")
    # Create a filepath directory string and export  
    # the aforementioned Julia graph into the filepath directory.
    currwd = os.getcwd()
    filepath = str(currwd) + "//JULIA_GRAPHS//{}.png".format(filename)
    plt.savefig(filepath, dpi = "figure", format = "png", bbox_inches = "tight", pad_inches = 0)
    # To avoid too much memory being used up, close the active pyplot window.
    plt.close()
    return None

Now, we shall create a function that exports `n_img` images from the Julia graph for $j_p = re^{ai}$, where $r$ is determined by the input of the function and $a$ is chosen at equally spaced intervals between `a_min` and `a_max`.

In [11]:
def julia_export(r, max_n, c_min, c_max, c_intervals, 
plt_width, plt_height, n_img, a_min, a_max, verbose = False):
    '''
    The julia_export function exports n_img number of images for 
    a Julia set with parameter j_p = re^{ai} where a is chosen
    at equally spaced intervals between a_min and a_max.
    '''
    # Calculate the width of the intervals between a_min and a_max.
    n_intervals = (a_max - a_min) / (n_img - 1)
    # Iterate i n_img times, for which n_img is the number of images.
    for i in range(n_img):
        # Calculate a in between the intervals and calculate j_p.
        a = a_min + (i * n_intervals)
        j_p = r * np.exp(a * 1j)
        # Create a filename and export an image of each Julia set.
        filename = "JuliaPic{}".format(i + 1)
        julia_graphpic(j_p, max_n, c_min, c_max, c_intervals, plt_width, plt_height, filename)
        # If verbose is True, output the current progress line as an addendum.
        if verbose:
            print("Please wait while the images are being exported. {}% of all the images has now been exported.".format(int((i + 1) / n_img * 100)), end = "\r")
    return None

In [12]:
def julia_animation(r = 0.7885, max_n = 100, c_min = -2, c_max = 2, 
c_intervals = 200, plt_width = 14, plt_height = 12, n_img = 200, 
a_min = 0, a_max = 2 * np.pi, screen_w = 800, screen_h = 600, verbose = True):
    '''
    The julia_animation function creates a PyGame/SDL animation with
    slides of images of the Julia graph with parameter j_p = re^{ai}, 
    where a is n_img - 1 equally spaced intervals between a_min and a_max.
    The animation has a resolution width of screen_w pixels and height of
    screen_h pixels.
    '''
    try:
        julia_export(r, max_n, c_min, c_max, c_intervals, plt_width, plt_height, n_img, a_min, a_max, verbose)
        if verbose:
            print("\nAll the images has now been exported. Starting Program...")
    except:
        print("Error producing graphs!")
        return None
    # Store the input screen size as a tuple.
    screen_size = (screen_w, screen_h)
    # Store colours as a RGB tuple.
    white = (255, 255, 255)
    # Set initial spped factor and load the time clock.
    speed_factor = 1
    clock = pygame.time.Clock()
    # Initialise PyGame.
    pygame.init()
    if verbose:
        print("Program has now initialised!")
    # Set the screen size and the title of the program.
    screen = pygame.display.set_mode(screen_size)
    title = "Animation of Julia Sets"
    title += "  "
    title += "(Keystroke:  \'Space\' to start or pause, \'Down\' to speed up and \'Up\' to slow down.)"
    pygame.display.set_caption(title)
    # Set the path of the Julia image and load the image to the correct size.
    currwd = os.getcwd()
    filename = str(currwd) + "//JULIA_GRAPHS//JuliaPic1.png"
    julia = pygame.image.load(filename)
    julia = pygame.transform.scale(julia, screen_size)
    # Create a rectangle for the image to be loaded in.
    julia_rect = pygame.Rect(0, 0, screen_w, screen_h)
    # Fill the background white and initialise the the image into the program.
    screen.fill(white)
    screen.blit(julia, julia_rect)
    # Refresh the program display.
    pygame.display.flip()
    # Set initial values of variables for later.
    Running = True
    pause = True
    i = 0
    # If Running is set to True, continue the program.
    while Running:
        # Obtain the events from system logs.
        for event in pygame.event.get():
            # If user quits the program, stop running the program.
            if event.type == pygame.QUIT:
                Running = False
            # If user presses the Space key, then stop or continue the animation.
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                pause = not pause
            # If animation is not paused, then allow functionality of speeding up or slowing down the animation.
            if not pause:
                # If the Down Arrow key is pressed, then slow down the animation by a fixed speed factor.
                if event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                    speed_factor = 0.75 * speed_factor
                # If the Up Arrow key is pressed, then speed up the animation by a fixed speed factor.
                elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                    speed_factor = (4 / 3) * speed_factor
        # Continue the animation if pause is False.
        if not pause:
            # If Running is false, then stop the program. This was mainly to stop freezing of the program when a user quits.
            if not Running:
                pygame.quit()
                return None
            # Set the timimg between the transition of images.
            fps = 10 * speed_factor
            clock.tick(fps)
            # Add the value of i by 1.
            i += 1
            # Once i becomes the maximum number of images, reset the value to 0.
            if i == n_img:
                i = 0
            # Create a filepath directory string based on the current
            # value of i and load the Julia graph corresponding to the
            # filepath directory.
            currwd = os.getcwd()
            filename = str(currwd) + "//JULIA_GRAPHS//JuliaPic{}.png".format(i + 1)
            julia = pygame.image.load(filename)
            # Resize the loaded Julia graph and update the display.
            julia = pygame.transform.scale(julia, screen_size)
            julia_rect = pygame.Rect(0, 0, screen_w, screen_h)
            screen.fill(white)
            screen.blit(julia, julia_rect)
            pygame.display.flip()
    # Close the PyGame window.
    pygame.quit()
    if verbose:
        print("Program has now exited.")
    return None

We shall now use the default parameters of the `julia_animation` function, which will generate $200$ image files of the Julia sets with parameter $j_p = 0.7885e^{ai}$ where the numbers $a$ are chosen at equally spaced intervals in $[0, 2 \pi]$, and display these image files in a film like animation with the program resolution being $800$ pixels by $600$ pixels.

In [13]:
julia_animation()

Please wait while the images are being exported. 100% of all the images has now been exported.
All the images has now been exported. Starting Program...
Program has now initialised!


From experimenting with the parameter `r` in the `julia_animation` function, one can see that changing the value of $r$ in $j_p = re^{ai}$ changes the distance between multiple "clusters" in the image, and it is only when these "clusters" meet that a significant visual effect happens. Upon trial and error, another value of $r$ that creates striking effects is $1.2$, and from this, one can see that there are many values of $r$ that creates such visual effects.

In [ ]:
'''
TODO: New code cell for different values of r, more refined version as a movie
'''

<a id='S4.2'></a> 
#### 4.2 Animation of Mandlebrot Sets

Firstly, need to know as to what a Mandelbrot set is. The **Mandelbrot set** is defined as the values of $c$ for which the following recurrence relation is bounded for all $(z_n)_{n \geq 0}$:

$$
z_0 = 0  \quad \text{and} \quad z_n = z_{n-1}^2 + c, \quad \text{for } n = 1,2,3,\ldots
$$

We say that $(z_n)_{n \geq 0}$ is the Mandelbrot sequence induced by $c$. From this, we can see that $(z_n)_{n \geq 0}$ only relies on $c$.

In [14]:
def mandelbrot(c, max_n):
    '''
    The mandelbrot function calculates the largest term n_max
    for which each term z_n of the corresponding Mandelbrot 
    sequence induced by c is bounded by 2. 
    '''
    # Set the "zeroth" term as 0.
    z =  0 + 0j
    # Set the largest number as 0.
    n_max = 0
    # Iterate until term is not bounded by 2.
    while abs(z) <= 2:
        # Derive the next term of the sequence from the current term.
        z = (z ** 2) + c
        # Increment the largest number n_max by 1.
        n_max += 1
        # Break the loop early when the largest number n_max reaches 
        # the maximum number of iterations max_n.
        if n_max == max_n:
            break
    # Return the value of the largest number n_max.
    return n_max

We shall now create data for our Mandelbrot set. In order to do this, we need to create vectors `x_vector` and `y_vector` in order to create grid matrices `x_matrix` and `y_matrix` as input matricies. The `x_vector` must be a number `n_xpts` of equally spaced intervals between the values `x_min` and `x_max`, and the `y_vector` must be a number `n_ypts` of equally spaced intervals between the values `y_min` and `y_max`.

We shall create the following function that takes a number `n_xpts` of equally spaced intervals between the values `x_min` and `x_max`, and takes a number `n_ypts` of equally spaced intervals between the values `y_min` and `y_max` and calculates `x_matrix` and `y_matrix`, and inputs the matrices into the `mandelbrot` function to create an output matrix `z_matrix`. In order to generate data needed for the set, we need `x_matrix`, `y_matrix` and `z_matrix`.

In [15]:
def mandelbrot_data(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts):
    '''
    The mandelbrot_data function calculates the matrices 
    required for graphing the mandelbrot function.
    '''
    # Calculate the vectors required for the two matrices required for input.
    x_vector = np.linspace(x_min, x_max, n_xpts)
    y_vector = np.linspace(y_min, y_max, n_ypts)
    x_matrix, y_matrix = np.meshgrid(x_vector, y_vector)
    # Create a zero output matrix with the same dimensions as the input matrices.
    x_matrix_rows, x_matrix_cols = np.shape(x_matrix)
    z_matrix = np.zeros((x_matrix_rows, x_matrix_cols), dtype = int)
    # Calculate every element of the output matrix for every 
    # element in the input matrix using the mandelbrot function.
    for i in range(x_matrix_rows):
        for j in range(x_matrix_cols):
            c = x_matrix[i, j] + y_matrix[i, j] * 1j
            z_matrix[i, j] = mandelbrot(c, max_n)
    return x_matrix, y_matrix, z_matrix

We shall create a folder, so that we can export the images into an appropriate directory entitled "MANDELBROT_GRAPHS".

In [16]:
# Obtain the current working directory.
currwd2 = str(os.getcwd())
# Create a subdirectory within the current working directory using the system terminal.
createdir2 = "mkdir \"{}//MANDELBROT_GRAPHS\"".format(currwd2)
os.system(createdir2)

1

We shall now create a function that generates and exports the Mandelbrot set.

In [17]:
def mandelbrot_graphpic(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts, fig_w, fig_h, filename):
    '''
    The mandelbrot_graphpic function creates a Mandelbrot graph and exports
    the Mandelbrot graph as a picture with default resuloutions.
    '''
    # Calculate the two input matrices and one output matrix using the mandelbrot_data function.
    x_matrix, y_matrix, z_matrix = mandelbrot_data(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts)
    # Produce a Mandelbrot graph from the given matrices without the axes.
    plt.figure(figsize = (fig_w, fig_h))
    plt.pcolor(x_matrix, y_matrix, z_matrix, cmap = 'plasma', vmin = 0, vmax = max_n, shading = "auto")
    plt.axis("off")
    # Create a filepath directory string and export  
    # the aforementioned Mandelbrot graph into the filepath directory.
    currwd = os.getcwd()
    filepath = str(currwd) + "//MANDELBROT_GRAPHS//{}.png".format(filename)
    plt.savefig(filepath, dpi = "figure", format = "png", bbox_inches = "tight", pad_inches = 0)
    # To avoid too much memory being used up,  
    # close the active pyplot window.
    plt.close()
    return None

Before we create a function that zooms in and out of a Mandelbrot set, we need to create a "recipe" for the function.

We shall now define as to what "clicking and dragging" the mouse is in terms of the actions.

> **Definition:** A click and drag of a mouse button consists of the following three actions in succession.
>
> 1. Pressing down on the left mouse button,
>
> 2. Moving the mouse, and
>
> 3. Releasing the left mouse button.
>
> All of these are stipulated by different events in PyGame.

Hence, for our program, the "recipe" for a "cropbox" is of the following:

> 1. If the user presses down the left mouse button, then set drag to be True and obtain position of the mouse (initial position).
>
> 2. If the user then moves the mouse around, and if Drag is True, then set rect to be True.
> 
> 3. If rect is True, then obtain current poisition of the mouse whenever the mouse moves around and display an outline box based on the initial position of the mouse and the current position of the mouse.
>
> 4. If the user releases the mouse button, then set rect and drag to be False.

PyGame has no definitive function for a button, hence we shall need a "recipe" for the buttons in our program, and the "recipe" is of the following:

> 1. Create a filled box with a shade of green, and position the box with appropriate co-ordinates.
>
> 2. If the user presses down the left mouse button within the co-ordinates of the button, set button variable to be True, and update the colour of the button to be a lighter green.
>
> 3. Once the processes initiated by the user or by the button has been completed and the button variable is set to False, the colour of the button will go back to the initial colour, that being green in this case.

Using the principles above, we shall now create a program that zooms in and out of the animation, and we shall call the associated function `mandelbrot_program`. Due to the textboxes from within the program, we must set the minimum compatibility resolution width as $680$ pixels.

In [18]:
def mandelbrot_program(max_n = 100, x_min = -2, x_max = 1, n_xpts = 400, y_min = -1.5, y_max = 1.5, n_ypts = 400, fig_w = 14, fig_h = 12, screen_w = 800, screen_h = 600):
    '''
    The mandelbrot_program function creates a PyGame/SDL program that
    displays a Mandelbrot graph, allowing the user to zoom in and out
    the graph. The animation has a resolution width of screen_w pixels
    and height of screen_h pixels.
    '''
    # Set the minimum width resolution for running the program.
    appcompatresx = 680
    if screen_w < appcompatresx:
        print("The width resolution you have inputted into the function is too small. You need a width resolution of at least {} pixels.".format(appcompatresx))
        return None
    # Try exporting the required images for the program.
    try:
        mandelbrot_graphpic(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts, fig_w, fig_h, "MandelbrotPic0")
    except:
        print("Error producing graphs!")
        return None
    # Store the input screen size and the size of the rescaled image as tuples.
    screen_size = (screen_w, screen_h)
    mandelbrot_size = (screen_w, screen_h - 50)
    # Store colours as RGB tuples.
    white = (255, 255, 255)
    black = (0, 0, 0)
    grey = (119, 119, 119)
    green_notactive = (114, 179, 101)
    green_active = (161, 251, 142)
    red = (255, 0, 0)
    orange = (240, 134, 80)
    # Set graph axes limits as lists and add initial values.
    x_min_list = []
    x_max_list = []
    y_min_list = []
    y_max_list = []
    x_min_list.append(x_min)
    x_max_list.append(x_max)
    y_min_list.append(y_min)
    y_max_list.append(y_max)
    # Initialise PyGame and its fonts.
    pygame.init()
    pygame.font.init()
    # Set the screen size and the title of the program.
    screen = pygame.display.set_mode(screen_size)
    title = "Animation of Mandelbrot Sets"
    title += " "
    title += '(Keystroke:  \'Drag and release\' to zoom in if the functionality is enabled.)'
    pygame.display.set_caption(title)
    # Set the path of the Mandelbrot image and load the image to the correct size.
    currwd = os.getcwd()
    filename = str(currwd) + "//MANDELBROT_GRAPHS//MandelbrotPic0.png"
    mandelbrot = pygame.image.load(filename)
    mandelbrot = pygame.transform.scale(mandelbrot, mandelbrot_size)
    # Create and position initial rectangles for Mandelbrot images, drag boxes, and zoom buttons.
    mandelbrot_rect = pygame.Rect(0, 0, mandelbrot_size[0], mandelbrot_size[1])
    cropbox = pygame.Rect(0, 0, 0, 0)
    zoom_in_button_left = screen_size[0] / 2 - 115
    zoom_out_button_left = screen_size[0] - 115
    zoom_button_top = screen_size[1] - 45
    zoom_button_width = 80
    zoom_button_height = 40
    zoom_in_button = pygame.Rect(zoom_in_button_left, zoom_button_top, zoom_button_width, zoom_button_height)
    zoom_out_button = pygame.Rect(zoom_out_button_left, zoom_button_top, zoom_button_width, zoom_button_height)
    # Create and position textboxes containing text.
    try:
        # Attempt to use the Segoe UI font by Microsoft(R).
        text = pygame.font.Font("segoeui.ttf", 14)
        errortext = pygame.font.Font("segoeui.ttf", 18)
    except:
        # Use the system Arial font by Monotype(R).
        text = pygame.font.SysFont("arial", 14)
        errortext = pygame.font.SysFont("arial", 18)
    text1 = text.render("Click the first button on the right", True, black)
    textbox1 = text1.get_rect()
    textbox_centre_x = zoom_in_button_left / 2
    textbox1_centre_y = screen_size[1] - 35
    textbox1.center = (textbox_centre_x, textbox1_centre_y)
    text2 = text.render("to enable the zoom in function.", True, black)
    textbox2 = text2.get_rect()
    textbox2_centre_y = screen_size[1] - 15
    textbox2.left = textbox1.left
    textbox2.centery = textbox2_centre_y
    text3 = text.render("to disable the zoom in function.", True, black)
    textbox3 = text3.get_rect()
    textbox3.left = textbox1.left
    textbox3.centery = textbox2_centre_y
    text4 = text.render("Click the button on the right to", True, black)
    textbox4 = text4.get_rect()
    textbox4.right = zoom_out_button_left - (zoom_in_button_left - textbox1.right)
    textbox4.centery = textbox1_centre_y
    text5 = text.render("zoom out to the previous image.", True, black)
    textbox5 = text5.get_rect()
    textbox5.left = textbox4.left
    textbox5.centery = textbox2_centre_y
    text6 = text.render("Please wait while", True, black)
    textbox6 = text6.get_rect()
    textbox6.center = textbox4.center
    text7 = text.render("it is zooming out.", True, black)
    textbox7 = text7.get_rect()
    textbox7.left = textbox6.left
    textbox7.centery = textbox5.centery
    # Set initial values of variables for later.    
    running = True
    drag = False
    rect = False
    zoomin_button = False
    zoomin = False
    zoomout_button = False
    zoomout = False
    i = 0
    # If running is set to True, continue the program.
    while running:
        # Obtain the events from system logs.
        for event in pygame.event.get():
            # If user quits the program, stop running the program.
            if event.type == pygame.QUIT:
                running = False
            # If user down clicks the left mouse button, then look for position and check status of buttons.
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    if event.pos[1] <= mandelbrot_size[1] and zoomin_button:
                        # If the position of the down click is within the image,
                        # then enable dragging and obtain the position of the mouse.
                        drag = True
                        mouse_x1_pos = event.pos[0]
                        mouse_y1_pos = event.pos[1]
                    # If the position of the down click is within region of the
                    # buttons, then set or change the activity statuses of the buttons.
                    elif event.pos[0] >= zoom_in_button_left and event.pos[0] <= (zoom_in_button_left + zoom_button_width) and event.pos[1] >= zoom_button_top and event.pos[1] <= (zoom_button_top + zoom_button_height):
                        zoomin_button = not zoomin_button
                    elif event.pos[0] >= zoom_out_button_left and event.pos[0] <= (zoom_out_button_left + zoom_button_width) and event.pos[1] >= zoom_button_top and event.pos[1] <= (zoom_button_top + zoom_button_height):
                        zoomout_button = True
            # If user releases the left click button, then set variables where necessary.      
            elif event.type == pygame.MOUSEBUTTONUP:
                if event.button == 1:
                    # Disable dragging.
                    drag = False
                    # If the zoom in functionality is enabled and rect is True, then set zoomin as True.
                    if zoomin_button and rect:
                        zoomin = True
                    # If the zoom out button is being clicked, then set zoomout as True.
                    elif zoomout_button:
                        zoomout = True
                    # Stop drawing the rectangle.    
                    rect = False
            # If the zoom in funtionality is enabled and dragging is enabled,
            # and the mouse is moving, then obtain the position of the mouse 
            # and set Rect as True.
            elif event.type == pygame.MOUSEMOTION and drag and zoomin_button:
                (mouse_x2_pos, mouse_y2_pos) = pygame.mouse.get_pos()
                rect = True
        # Fill the background with white.
        screen.fill(white)
        # Depending on the activity status of the buttons, draw
        # the appropriate textboxes and buttons onto the screen.
        screen.blit(text1, textbox1)
        if zoomin_button:
            screen.blit(text3, textbox3)
            pygame.draw.rect(screen, green_active, zoom_in_button)
        else:
            screen.blit(text2, textbox2)
            pygame.draw.rect(screen, green_notactive, zoom_in_button)
        if zoomout_button:
            pygame.draw.rect(screen, green_active, zoom_out_button)
        else:
            screen.blit(text4, textbox4)
            screen.blit(text5, textbox5)
            pygame.draw.rect(screen, green_notactive, zoom_out_button)
        # If rect is True, then calculate the appropriate position and 
        # size of the rectangle and draw the rectangle onto the screen.
        if rect:
            rect_width = abs(mouse_x2_pos - mouse_x1_pos)
            if mouse_y2_pos <= mandelbrot_size[1]:
                rect_height = abs(mouse_y2_pos - mouse_y1_pos)
            rect_left = min(mouse_x1_pos, mouse_x2_pos)
            rect_top = min(mouse_y1_pos, mouse_y2_pos)
            cropbox = pygame.Rect(rect_left, rect_top, rect_width, rect_height)
            screen.blit(mandelbrot, mandelbrot_rect)
            pygame.draw.rect(screen, grey, cropbox, 2)
            pygame.display.flip()
        else:
            # If zoomin is True, then start zooming in.
            if zoomin:
                # Display the appropriate message onto the screen.
                textwait = errortext.render("Please wait while the image is zooming in...", True, white, orange)
                textboxwait = textwait.get_rect()
                textboxwait_centre_x = mandelbrot_size[0] / 2
                textboxwait_centre_y = mandelbrot_size[1] / 2
                textboxwait.center = (textboxwait_centre_x, textboxwait_centre_y)
                screen.blit(mandelbrot, mandelbrot_rect)
                # Display the current rectangle onto the screen.
                pygame.draw.rect(screen, grey, cropbox, 2)
                screen.blit(textwait, textboxwait)
                pygame.display.flip()
                # Linearly interpolate the screen resolution of the rectangle against the axes of
                # the corresponding Mandelbrot graph, and add the new axes limits to the list.
                y_diff = y_max - y_min
                x_diff = x_max - x_min
                y_max_old = y_max
                x_min_old = x_min
                y_max = y_max_old - ((rect_top / mandelbrot_size[1]) * y_diff)
                rect_bottom = rect_top + rect_height
                y_min = y_max_old - ((rect_bottom / mandelbrot_size[1]) * y_diff)
                x_min = x_min_old + ((rect_left / mandelbrot_size[0]) * x_diff)
                rect_right = rect_left + rect_width
                x_max = x_min_old + ((rect_right / mandelbrot_size[0]) * x_diff)
                x_min_list.append(x_min)
                x_max_list.append(x_max)
                y_min_list.append(y_min)
                y_max_list.append(y_max)
                # Store the Mandelbrot graph with new axes limits
                # into a new image file, and display the new image.
                i += 1
                mandelbrot_graphpic(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts, fig_w, fig_h, "MandelbrotPic{}".format(i))
                filename = str(currwd) + "//MANDELBROT_GRAPHS//MandelbrotPic{}.png".format(i)
                mandelbrot = pygame.image.load(filename)
                mandelbrot = pygame.transform.scale(mandelbrot, mandelbrot_size)
                screen.blit(mandelbrot, mandelbrot_rect)
                pygame.display.flip()
                # Clear the system events log to avoid bugs.
                pygame.event.clear()                
                # Set zoomin to False.
                zoomin = False
            elif not zoomout:
                # If zoomin and zoomout is False, then draw 
                # the current Mandlebrot graph onto the screen.
                screen.blit(mandelbrot, mandelbrot_rect)
                pygame.display.flip()
            # If zoomout is True, then start zooming out where necessary.
            if zoomout:
                # Check image index.
                if i <= 0:
                    # Display an error message for one second when user has zoomed out to the maximum.
                    texterror1 = errortext.render("ERROR: You have zoomed out to the original image!", True, white, red)
                    textboxerror1 = texterror1.get_rect()
                    textboxerror1_centre_x = mandelbrot_size[0] / 2
                    textboxerror1_centre_y = mandelbrot_size[1] / 2
                    textboxerror1.center = (textboxerror1_centre_x, textboxerror1_centre_y)
                    screen.blit(mandelbrot, mandelbrot_rect)
                    screen.blit(texterror1, textboxerror1)
                    pygame.display.flip()
                    zoomout = False
                    zoomout_button = False
                    pygame.time.delay(1000)
                    pygame.event.clear()
                else:
                    # Display the appropriate message onto the screen.
                    i -= 1
                    screen.blit(text6, textbox6)
                    screen.blit(text7, textbox7)
                    screen.blit(mandelbrot, mandelbrot_rect)
                    pygame.display.flip()
                    # Display the previous zoomed out image, and go back to the previous axes limits.
                    filename = str(currwd) + "//MANDELBROT_GRAPHS//MandelbrotPic{}.png".format(i)
                    mandelbrot = pygame.image.load(filename)
                    mandelbrot = pygame.transform.scale(mandelbrot, mandelbrot_size)
                    x_min = x_min_list[i]
                    x_max = x_max_list[i]
                    y_min = y_min_list[i]
                    y_max = y_max_list[i]
                    x_min_list.pop()
                    x_max_list.pop()
                    y_min_list.pop()
                    y_max_list.pop()
                    # Clear the system events log to avoid bugs.
                    pygame.event.clear()
                    # Set zoomout to False, and "unclick" the zoom out button.
                    zoomout = False
                    zoomout_button = False
    # Close the PyGame window.
    pygame.quit()
    return None

We shall now call the function with the default variables. This in turn produces a program with resolution of $800$ pixels by $600$ pixels, with the initial Mandelbrot set within the region $[-2, 1] \times [-1.5, 1.5]$. 

In [19]:
mandelbrot_program()

<a id='S5'></a>
### Section 5: Tower of Hanoi 

TODO: Add introduction to ToH.

<a id='S5.1'></a> 
#### 5.1 Definitions and Objectives


In this section, we shall look at the necessary definitions and code to describe our objectives accurately in terms of the output of the code.

Firstly, we need to make some key definitions. Note that we shall call the pegs from left to right pegs $A$, $B$ and $C$ respectively.


> **Definition**: In the puzzle, A **move** consists of the following two pieces of information:
>
> •	A positive integer $d$ as the disc (of size $d$) being moved, and
>
> •	An indicator of the direction of the move, for which the values can only be Left or Right.

> **Definition**: A **peg** is a list of discs. A **configuration** (of the pegs) is the ordered triple $T =  (A, B, C)$ of the pegs mentioned thereof.

Remark: We shall explain further what an "indicator" means in this context when we implement the class Move in the next section.


Next, we need to know mathematically what a puzzle is.

> **Definition**: A **puzzle** $P$ consists of the following three pieces of information:
>
> •	A configuration called an **initial configuration** $T_0$,
>
> •	A configuration called an **objective configuration** $T_f$, and
>
> •	A set of Rules $R$.

Different variants of the Towers of Hanoi has different initial configurations, objective configurations, and rules.


We shall now introduce a rigorous definition of solving a puzzle.

> **Definition**: Given a puzzle, a move $M$ and a configuration $T$, $M$ is **a valid move of $T$ under $P$** if the rules $R$ in $P$ are followed.
>
> For such a move, the configuration $T$ shall be changed accordingly to T′, which we shall call $T’$ as **the configuration attained by $M$ on $T$** (under $P$).

> **Definition**: For a positive integer n, a **solution for the puzzle $P$ with $n$ disks** is a sequence of moves $<M_1, M_2, …, M_s>$ such that the following holds under $P$:
>
> 1. $T_i$ is the configuration attained by $M_i$ on $T_{i-1}$ ,
> 2. $M_i$ is a valid move of $T_{i - 1}$,
> 3. $T_s$ is the objective configuration of the puzzle $T_f$,
>
> for all $1 ≤ i ≤ s$. We shall call $s$ the **length** of the solution. This gives rise to a configuration list under a solution for the puzzle $<T_1, T_2, …, T_s>$.


Using the terminology, we can state our objectives rigorously.

TODO: Add references here

> Objective: Given the number of discs $n$, we should find a solution and a configuration list of the following puzzles:
>
> •	$P$, the Classical Tower of Hanoi Problem,
>
> •	$P$, the No Wrap Variant Problem, where wrap-around is prohibited, and
>
> •	$P$, the Bicolor Variant Problem, where the task is to separate 2 colours of discs.

Note that the output, namely the solution and the configuration list, shall be useful when we animate the problem in Part (reference here please).

It would be convenient for us to define $M$ and $T$ as a type of their own, as it would be easier to produce solutions and configuration lists in Python in that we can realize them as a `list` of a particular type.


We shall initialise $T$ as a list of lists. The structure of the tower is also noticeably clear from this. Lists are also mutable; hence it shall be easier for us to change the content later.

In contrast, a move does not have to be mutable. Thus, it would be beneficial to exploit the object-oriented programming aspect of Python (reference here please). Abiding by the definition, we shall define a new class Move that contains the two aspects of disc and direction.

It is now easier to determine the following types of solutions that we are looking for:

-	A Solution: A `list` of Type `Move`, and

-	A configuration list under a solution: a `list` of type `int list X int list X int list`, where `X` denotes the Cartesian product.

From now on, we shall assume that everything shall be of the right type. That is, $n$ shall automatically denote the number of discs by default, and a solution and configuration shall always have the aforementioned type. This is a general programming concept called *duck typing*, and it stems from the following claim:

> If something walks and quacks like a duck, then it must be a duck. 
>
> (“Understanding Interfaces in Go – Duncan Leung”)

This is a *pythonic* design, for which the concept is explained in their documentation in (Python Software Foundation, 2001).

<a id='S5.2'></a> 
#### 5.2 Producing a Solution

Using the above, we shall create a function that solves the puzzles using a powerful technique called **mutual recursion**. We have seen a lot of examples of simple recursion in lectures, but to produce the required solution for this case, we need to harness the power of mutual recursion. That is, we use multiple functions that recursively call each other.

TODO: Reference

Mutual recursion is a powerful idea, and there are many practical applications. One can see (reference here please) for more examples.

In particular, the following pseudocode illustrates the idea. Note that `MoveLeft(n)` produces the solution we need since the objective of the puzzle is to move the discs from peg A to C.

```
DEFINE MoveLeft(n): \\ Solution for the classical Puzzle - Move n discs from peg A to C
    if n = 1 \\ Base Case
        return [Move (1, 'Left')]
    else:
        \\ Perform the Following 3 Steps in order:
        1. MoveRight(n-1) \\ Move the first n-1 discs from peg A to B
        2. Move(n, 'Left') \\ Move disc n from peg A to C
        3. MoveRight(n-1) \\ Move the first n-1 discs from peg B to Peg C
END

DEFINE MoveRight(n): \\ Solution for moving n discs from peg C to A
    if n = 1 \\ Base Case
        return [Move(1, 'Right')]
    else:
        \\ Perform the Following 3 Steps in order:
        1. MoveLeft(n-1) \\ Move the first n-1 discs from peg A to C
        2. Move(n, 'Right') \\ Move disc n from peg A to B
        3. MoveLeft(n-1) \\ Move the first n-1 discs from peg C to Peg B
END
```

As seen, the function `MoveLeft` and `MoveRight` recursively call each other, this is an idea we shall keep on using. Now we shall see how it is implemented in Python.

Note that, other than the recursive call, memoization is implemented. This drastically improves the running time of the command, especially on large $n$. We will look at the space/time complexity issue in the last section.

Now let us see the solution to the puzzle with $n = 4$:


This is not particularly useful, because every element had been masked as an object. In order to see the moves, we can use a for loop:

So, we have produced our solution for the classical puzzle as required. We can also investigate the length of the solution:

We note that is not hard to show the (optimal) length for this classical version of the puzzle is always $2^n - 1$ (Miodrag Petković, 2009).

TODO: Maybe have my own document? Also maybe put this in extension or mention it again.

We shall now look at different variants of the puzzle. Firstly, we shall deal with the variant where no wrapping is allowed. Another equivalent way of forming this puzzle would be considering the puzzle as the classical variation of the Tower of Hanoi puzzle, with an additional rule that *every move must involve peg $B$*.

We shall also handle the problem with mutual recursion with a pseudocode algorithm; this one is particularly inspired by the discussion at (reference here please).

```
DEFINE NoWrap_Left(n): \\ Solution for the puzzle - Move n discs from peg A to C
    if n = 1: 
        \\ Perform the following 2 steps in order:
        1. Move(1, 'Right') \\ move disc from A to B
        2. Move(1, 'Right') \\ move disc from B to C
    else:
        \\ Perform the 5 Steps in Order:
        1. NoWrap_Left(n) \\ move n-1 from A to C 
        2. Move(n, 'Right') \\ move disc n from A to B
        3. NoWrap_Right(n) \\ move n-1 from C to A 
        4. Move(n, 'Right') \\ move disc n from B to C
        5. NoWrap_Left(n) \\move n-1 from A to C 
END    

DEFINE NoWrap_Right(n): \\ Solution for Moving n discs from peg C to A
    if n = 1:
        \\ Perform the following 2 steps in order:
        1. Move(1, 'Left') \\ move disc from C to B
        2. Move(1, 'Left') \\ move disc from B to A
    else:
        \\ Perform the 5 Steps in Order:
        1. NoWrap_Right(n) \\ move n-1 from C to A 
        2. Move(n, 'Left') \\ move disc n from C to B
        3. NoWrap_Left(n) \\ move n-1 from A to C 
        4. Move(n, 'Left') \\ move disc n from B to A
        5. NoWrap_Right(n) \\move n-1 from C to A 
END
```
The implementation is analogous to the classical variant of the puzzle. We shall also have a look at the solution and its length.


And, in fact, we can show that the (optimal) length of this solution is $3^n -1$ (reference here please). The intuition would be that we use one extra move in every iteration to accommodate the fact that it must pass through peg B.

Finally, we shall deal with the bicolour variant. Note that this implementation is not optimal, as we would see in the later section.

The strategy here is to first move everything to peg $B$ and then redistribute it after changing the bottom two discs. Again, we shall start by introducing pseudocode:

```
DEFINE ToCenter(n): \\ Solution for Moving n discs to peg B
    if n = 1:  Move(1, 'Right') 
    Elif n = 2: [Move(2, 'Left'), Move(1, 'Right')]
    else:
        if n is even:
            \\ Perform the 4 Steps in Order:
            1. ToCenter(n-2) \\ move n-2 discs to peg B
            2. Left(n-2) \\ move n-2 discs from B to A
            3. Move(n, 'Left') \\ move n from C to B 
            4. Right(n-1) \\move n-1 discs from A to B 
        else: \\ if n is odd
            \\ Perform the 4 Steps in Order:
            1. ToCenter(n-1) \\ move n-1 discs to peg B
            2. Right(n-1) \\ move n-1 discs from B to C
            3. Move(n, 'Right') \\ move n from A to B 
            4. Left(n-1) \\move n-1 discs from C to B 
END    

DEFINE Solve_Bicolor(n): \\ Solution for the Bicolor Variant
    if n is not even, RAISE error
    if n = 2: [Move(2, 'Left'), Move(1, 'Left'), Move(2, 'Left')]
    else:
        \\ Perform the 3 Steps in Order:
        1. ToCenter(n-1) \\ move n-1 discs to peg B
        2. Move(n, 'Left') \\ move disc n from C to A
        3. REVERSE ToCenter(n) \\ redistribute the n-1 discs to their opposite pegs
END
```

Note that, for the last step, we shall need to reverse the whole list. In this case, we shall need to create a new function `rev_list` to reverse a list.

<a id='S5.3'></a> 
#### 5.3 Identifying Valid Moves

In this section, we shall introduce the function `ToH_Valid_Move` in order to detect whether a move $M$ is a valid move of $T$ under $P$. We shall then use it to return the configuration list under a solution. This will achieve the objective set in the first part of our discussion. From here, we shall also present an interactive version of the puzzle.

Let us understand what the final goal is before presenting the code.

The key idea of the ToH_Valid_Move Command is as follows:

>1.	Given a move `m`, assign a direction of m (i.e., `m.direction`) as Left or Right with a direction number `dir_num`, with the value being `1` when the direction is Right, and `-1` otherwise.
> 2.	Given the Move `m,` ensure that the disc m.disc is at the top of one of the pegs. Find the peg and call that peg `count`.
> 3.	Calculate the value of `count + dir_num` and assign it to `tar`. This tells us where we can put the disc at the target configuration. If wrapping around is allowed, we can work modulo 3; otherwise `tar` can only be 0, 1 or 2.
> 4.	Mutate the configuration such that it becomes the configuration attained by `m` on `config`.


The key point here is point 4. Due to the process of mutating the original configuration `config`, it will change whenever we call the function, so we do not have to redefine it upon successive calls of the function. 

This is an immensely powerful design (in technical jargon, he variable `config` is *dynamic*) that will save us some lines when designing the interactive version of the puzzles. The following code block illustrates this idea.

From here, we shall display the configuration list under a solution for the puzzle. We shall first start with the classical variant.

We can deal with the no wrap variant and bicolor variant similarly.

As we can see, the answer of the configuration list indeed matches the definition and the objective configuration. Note that, in the `ToH_BiColor_Solve` function, the output is sub-optimal. For example, note that the output, shown below, after 4 steps,

```
[[1, 3], [], [2, 4]] <-- HERE
Step 1: move disc 2 to the Left
[[1, 3], [2], [4]]
Step 2: move disc 1 to the Right
[[3], [1, 2], [4]]
Step 3: move disc 1 to the Left
[[1, 3], [2], [4]]
Step 4: move disc 2 to the Right
[[1, 3], [], [2, 4]] <-- HERE
```

brings us back to the original configuration, so this does not need to be part of the solution.

From here, it is not hard for us to produce an interactive version of the three puzzles, where the user can feed in a move and play until the objective is reached.

<a id='S5.4'></a> 
#### 5.4 Animation of the Puzzles

In this section, we shall look as to how we can integrate the solution and configuration list such that the puzzles can be displayed as an animation. We also explore more complicated animations.

First we shall create a background with a base and 3 pegs, just like the usual Tower of Hanoi setup.

Now, in order to access and update the background, we just need to use the image, load and bilt command in the PyGame module.

Note that we do not want to do the same and save discs as PNG files, and this is because they depend on the configuration, and they will be moved around in the animation.


Another hurdle to overcome is the fact that discs come in varied sizes. Usually, they are cylinders of the same height, but since we are looking at the puzzle from a side view, we shall treat the discs as rectangles with different widths but the same height.

In our implementation, the discs are labelled by the positive integers (which stemmed from the class `Move`), and we can set disc $1$ as $10$ pixels long, disc $2$ as $20$ pixels, ... et cetera.


If the disc size is fixed, where the discs are placed is uniquely determined by the configuration. We shall take advantage of this and write a function that, given the configuration, outputs where the discs should be drawn.

(Note that the `draw` command in `PyGame` only takes from the top left corner TODO: What?, hence the co-ordinates that we return for each disc will be from the top left corner.)


The `ToH_Get_Coordinate` acts as a helper to our animation functions, and we shall enumerate over the list to draw all the discs in a configuration.

(Note that the animation shows the configuration before applying the last move, thus it is easy to see that the animation produces the objective configuration.)

Now we shall discuss the key difference between implementing this and the other two variants of the puzzle.

1.	The No Wrap Variant is similar, except with WrapAround = False in the ToH_Valid_Move command.

2.	The Bicolor Variant: The initial configuration of the discs shall be changed accordingly. We shall also colour the discs red and green instead of blue for visual effects.


From here, we can let the user choose the mode, discs, and speed factor using an interactive command.

<a id='S5.5'></a> 
#### 5.5 Extensions

There are a lot of features we can add to the functions introduced in this section. Firstly, we look at implementing a smooth version of ToH_Basic, where the discs transfer smoothly.

Now as promised in the previous section, we look at different time and space complexity issue. 

TODO: Some comment on space complexity

We shall look at how memoization speeds up the function call for large values of $n$. 


We first start by implementing the no memoization version of functions that produces a solution, with a suffix `_NoMem` after their usual names in order to distinguish them.


As seen, as $n$ increases, the run time for no memoization dramatically increases, but with memoization, the running time can be kept to around 400 nanoseconds.

TODO: Graphical Representation of this fact?

TODO: Iterative design on the functions?

<a id='Conc'></a>
## Conclusion 

Throughout the project, we have explained the complex topic of fractals and we have seen examples of how mathematical concepts like recursion can be implemented using various computational techniques. As Mandelbrot himself (Benoît Mandelbrot, 1983) put it:
> *“Fractals Geometry reveals that some of the most austerely formal chapters of mathematics had a hidden face: a world of pure plastic beauty unsuspected till now.”*
>
> Benoît Mandelbrot, 1983


<a id='Bib'></a>
## References and Bibilography

<a id='PyGame_Intro'>(Shinners, 2021)</a> Shinners, P. (2021). *Pygame Intro — Pygame v2.1.1 Documentation.* [online] Pygame.org. Available at: http://www.pygame.org/docs/tut/PygameIntro.html [Accessed 5 May 2022].